In [88]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
from sklearn.utils import shuffle


## Load Data

In [89]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_1sec.csv')

### Remove Emotions from dataset & Split into Train/Test sets

In [90]:
dataset = dataset.drop(columns=['EMOTION_P', 'EMOTION_A', 'EMOTION_D'])

train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

x_train = train_dataset.reshape((len(train_dataset), np.prod(train_dataset.shape[1:])))
x_test = test_dataset.reshape((len(test_dataset), np.prod(test_dataset.shape[1:])))

print(len(x_train[0]))

No Training Samples: 3840
No Test Samples: 960
27


## Build Model

### Settings

In [91]:
latent_dim = 5
intermediate_dim = 12
input_shape = (dataset.shape[1],)

### Sampler

In [92]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        
        batch = tf.shape(z_mean)[0]
        
        dim = tf.shape(z_mean)[1]
        
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    
def sample_point_from_normal_distribution(args):
    mu, log_variance = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0.0, stddev=1.0)
    sampled_point = mu + K.exp(log_variance / 2) * epsilon
    return sampled_point

### Encoder

In [93]:
encoder_inputs = keras.Input(shape=input_shape)
x = layers.Dense(intermediate_dim, activation="relu")(encoder_inputs)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 27)]         0           []                               
                                                                                                  
 dense_8 (Dense)                (None, 12)           336         ['input_7[0][0]']                
                                                                                                  
 z_mean (Dense)                 (None, 5)            65          ['dense_8[0][0]']                
                                                                                                  
 z_log_var (Dense)              (None, 5)            65          ['dense_8[0][0]']                
                                                                                            

### Decoder

In [94]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)

decoder_outputs = layers.Dense(input_shape[0], activation="sigmoid")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense_9 (Dense)             (None, 12)                72        
                                                                 
 dense_10 (Dense)            (None, 27)                351       
                                                                 
Total params: 423
Trainable params: 423
Non-trainable params: 0
_________________________________________________________________


### Define VAE as a model with a custom train_step

In [112]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction)
                )
            )
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


## Train VAE

In [118]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(dataset, epochs=30, batch_size=128)

Epoch 1/50
120/120 [==============================] - 1s 2ms/step - loss: 39076.5664 - val_loss: 39076.3906
Epoch 2/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.6406 - val_loss: 39076.5586
Epoch 3/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.6641 - val_loss: 39076.4414
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 39076.5000 - val_loss: 39076.3633
Epoch 5/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.4492 - val_loss: 39076.3477
Epoch 6/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.4883 - val_loss: 39076.3711
Epoch 7/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.4336 - val_loss: 39076.3320
Epoch 8/50
120/120 [==============================] - 0s 1ms/step - loss: 39076.4453 - val_loss: 39076.3242
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 39076.4219 - val_loss: 39076.2070
Epoch 10/50
120/120 [=======

### Test

In [119]:
sample = np.asarray(dataset.iloc[1])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.32803145  0.16344059  0.23296083  0.32319513  0.4165411   0.45243546
   0.295855    0.32972974  0.286151    0.27292065 -0.17473445  0.00446567
  -0.33494135  0.92557049  0.22393783  0.10442776  0.03750376  0.1268874
   0.05702575  0.12193758  0.29428763  0.04635692  0.08562997  0.02835034
   0.12710496  0.28694534  0.06570215]]


In [122]:
z_mean, z_log_var, z = vae.encoder.predict(sample)

In [123]:
regen = vae.decoder.predict(z)
print(regen)

[[5.87805688e-01 2.93179452e-01 3.04016978e-01 5.21048129e-01
  4.11051273e-01 4.15717572e-01 3.14966768e-01 3.21503758e-01
  2.86111832e-01 2.75454938e-01 2.91687954e-38 8.32346082e-03
  1.12921836e-04 8.14915419e-01 3.39755714e-01 1.01985008e-01
  1.63266033e-01 4.14855570e-01 4.38008755e-01 3.27816665e-01
  3.71380925e-01 3.08105528e-01 6.63352489e-01 6.87188685e-01
  4.96380389e-01 5.69266737e-01 4.04612035e-01]]


In [10]:
def vae_loss(input_vols, output_vols):
    beta = 1e-7
    kl_loss = tf.keras.backend.sum(-1 - tf.keras.backend.log(tf.keras.backend.exp(z_log_var)) + tf.keras.backend.exp(z_log_var) + tf.keras.backend.square(z_mean))/2
    return tf.keras.backend.mean((input_vols-output_vols)**2) + beta*kl_loss

In [220]:
class VAE:
    def __init__(self, encoder, decoder, encoder_input, z_mean, z_log_var):
        self.encoder = encoder
        self.decoder = decoder
        
        self._model_input = encoder_input
        self.mu = z_mean
        self.log_variance = z_log_var
        
        self.model = None
        self._build_autoencoder()
        
        self.reconstruction_loss_weight = 1000
        
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()
        
    def compile(self, learning_rate=0.0001):
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                          loss=self._calculate_combined_loss,
                          metrics=[self._calculate_reconstruction_loss,
                                   self._calculate_kl_loss])
        
    def train(self, train_set, batch_size, num_epochs):
        self.model.fit(train_set, train_set,
                      batch_size = batch_size,
                      epochs=num_epochs,
                      shuffle=True)
        
    def reconstruct(self, samples):
        latent_representations = self.encoder.predict(samples)
        reconstructed = self.decoder.predict(latent_representations)
        return reconstructed, latent_representations
    

    
    def _build_autoencoder(self):
        model_output = self.decoder(self.encoder(self._model_input))
        self.model = keras.Model(self._model_input, model_output, name="AutoEncoder")
        return
    
    def _calculate_combined_loss(self, y_target, y_predicted):
        reconstruction_loss = self._calculate_reconstruction_loss(y_target, y_predicted)
        kl_loss = self._calculate_kl_loss(y_target, y_predicted)
        print("HIIIIIIIIIIII")

        combined_loss = (self.reconstruction_loss_weight * reconstruction_loss) + kl_loss
        print(combined_loss)
        return combined_loss
    
    def _calculate_reconstruction_loss(self, y_target, y_predicted):
        error = y_target - y_predicted
        reconstruction_loss = K.mean(K.square(error), axis=1) # this axis may be wrong
        
        return reconstruction_loss
    
    def _calculate_kl_loss(self, y_target, y_predicted):
        kl_loss = -0.5 * K.sum(1 + self.log_variance - K.square(self.mu) - K.exp(self.log_variance), axis=1)
        return kl_loss